# Task 1


For the given task of disambiguating protein sequences, we can are give assembly_1.prot.fa which is poorly annotated and contains query sequences. 

It must be mapped to the reference sequences contained in assembly_2.prot.fa. In this case, it is suitable to use a pairwise local sequence alignment, to find the similarity between a given "query" and "reference" or "subject" sequence, since the purpose of doing this is to get rid of redundancy between the assemblies, while still allowing us to identify some variations. 

The PairwiseAligner() function uses the Smith-Waterman algorithm which may already be optimised in Biopython's package. 

Though the filling up of matrices to calculate alignment scores is memory-intensive, we can also use protein substituion matrices such as BLOSUM62 similar to the blastp algorithm,  or input our own criteria for scoring matrices. 

Here only the top scored alignment was selected to give the final aligned sequence. This ensured that the disambiguation process only focused on the most relevant and significant alignments. 

Based on this top scoring alignment the query coverage and sequence identity was calculated to give us an idea of how similar the two sequences are. Query cover refers to the percentage of the query sequence that overlaps the reference sequence. Sequence identity refers to the percentage of base pairs that are the same between the query sequence and the reference sequence. The alignments which were not above the threshold were return as non - significant alignments.

    Query Coverage = Number of Non-Gap Positions in Aligned Query / Total Length of Query Sequence
    Identity= Number of Identical Positions / Aligned length
​
The query fasta file and subject fasta file were parsed using the SeqIO package in Biopython. The sequences in the query file were iterated one by one over all the sequences in the reference file and the aligned query sequences including gaps were written in an output fasta file, containing both the query and subject ids.


In [1]:
from Bio import Align
from Bio.Align import substitution_matrices

def pairwise_local_alignment(query_seq, subject_seq, query_coverage_threshold = 90, identity_threshold = 95):

    #remove dots from fasta sequence
    query_seq = query_seq.rstrip('.')

    #perform pairwise alignment
    aligner = Align.PairwiseAligner()
    aligner.mode = 'local'  # Set the alignment mode to 'local'
    #aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
    alignments = aligner.align(query_seq, subject_seq)

    #get top scoring alignment only
    if alignments:
        
        best_alignment = max(alignments, key=lambda x: x.score)
        aligned_query_seq = best_alignment[0]
        aligned_subject_seq = best_alignment[1]

        # calculate length, query coverage and sequence identity percentage
        aligned_length = sum(1 for a, b in zip(aligned_query_seq, aligned_subject_seq) if a != '-' and b != '-')
        query_coverage = aligned_length / len(query_seq) * 100
        identity = sum(a == b for a, b in zip(aligned_query_seq, aligned_subject_seq)) / aligned_length * 100

        # check if alignment meets our set thresholds
        if query_coverage >= query_coverage_threshold and identity >= identity_threshold:
            return aligned_query_seq, aligned_subject_seq, query_coverage, identity
            
    return None, None, 0, 0


In [ ]:
# running a test sequence:

#query_seq = "MPLLGIHCTDCCTAFVLFRRRLIQKLWPVVYFLYAFILEIEYVAWNNLFPKTSWLQVKPVCIATTAGGLELSNFMQHLMFYMHYLFSLHIQELGSILRLFFYSCLQELELTTAGPLLAISWCLCFTCFKLQADHVSSFSESSTYHKFLRKNSFVFI."
#subject_seq = "MNKMILEDPPNPAKIISKQLTKTDLVRNVKLPKKQTESVLTMMTGGTTENLQNGKEVKVLDYIEGNEYTVVLRCTENGKYHFGDGWSTMKHSLKLQEGEILSLYWDYKNQQFIII"


#aligned_query, aligned_subject, query_coverage, identity = pairwise_local_alignment(query_seq, subject_seq)

#if aligned_query and aligned_subject:
    #print(f"Aligned Query Sequence: {aligned_query}")
    #print(f"Aligned Subject Sequence: {aligned_subject}")
    #print(f"Query Coverage: {query_coverage:.2f}%")
    #print(f"Identity: {identity:.2f}%")

#else:
    #print("No significant local alignment found.")


In [2]:
def write_alignment_to_fasta(output_fasta, seq1_id, seq2_id, aligned_query_seq):
    # Write aligned query sequence to the output FASTA file
    output_fasta.write(f">{seq1_id}||{seq2_id}\n{aligned_query_seq}\n")

In [ ]:
from Bio import SeqIO

def align_sequences_from_files(file1, file2, output_file, query_coverage_threshold=90, identity_threshold=95):
    
    # open new FASTA file for writing matched sequences
    with open(output_file, "w") as output_fasta:
        
        # iterate over sequences in the first file
        for seq1 in SeqIO.parse(file1, "fasta"):
            
            # iterate over sequences in the second file
            for seq2 in SeqIO.parse(file2, "fasta"):
                
                # performing pairwise local alignment
                aligned_query_seq, aligned_subject_seq, query_coverage, identity = pairwise_local_alignment(seq1.seq, seq2.seq, query_coverage_threshold, identity_threshold)

                # check if alignment meets specified thresholds
                if aligned_query_seq and aligned_subject_seq:
                    
                    # write aligned query sequence to the output FASTA file
                    write_alignment_to_fasta(output_fasta, seq1.id, seq2.id, aligned_query_seq)


# batch set
align_sequences_from_files("1.fa", "assembly_2.prot.fa","test_output.fasta")

#actual set
align_sequences_from_files("assembly_1.prot.fa", "assembly_2.prot.fa","output.fasta")

An alternative solution to the problem is using CD-HIT in CLI. However this is only used to form clusters, not for accurate making alignments. To cluster sequences based on their similarity using the command: 


cd-hit -i assembly_1.prot.fa -d your_reference.fasta -o output_file -c 0.95 -aS 0.9

where the sequence identity threshold is 0.95 and the sequence coverage threshold is 0.90. Using this method it is fairly easy to use the fasta file format, specify thresholds using built in parameters, and output a fasta file containing clusters of similar sequences with the sequence ids of both query and reference assemblies.

This method is used to reduce redundancies which fulfills our initial purpose of disambiguating protein sequences. Clustering them may also be beneficial in grouping them based on functional protein domains and assist us in identifying hydrolase activity. 
